In [1]:
import json
import os
"""

DESC: For getting a target.json which only has the requires rewrite field.
"""
def process_and_save_json(input_path, output_path):
    # Read the JSON data from the input file
    with open(input_path, 'r') as file:
        data = json.load(file)

    # Loop through the data and process as required
    for dialog_key, dialog_data in data.items():
        dialog_data['annotator_id'] = None
        annotations = []
        key_to_remove = []
        for key, value in dialog_data.items():
            # Check if the key is a digit
            if key.isdigit() and key == "1":
                key_to_remove.append(key)
            elif key.isdigit():
                # Add the 'requires_rewrite' key
                annotations.append({'turn_num': int(key),'turn_id': dialog_data['dialog'][int(key)-1]['turn_id'] , 'requires rewrite': None})
                key_to_remove.append(key)
        
        dialog_data['annotations'] = annotations

        for key in key_to_remove:
            del dialog_data[key]

    # Write the modified data to the output file
    with open(output_path, 'w') as file:
        json.dump(data, file, indent=4)

def convert_all_json_batches(folderpath):
    batch_num = 1
    for file_name in os.listdir(folderpath):
        if file_name.endswith('.json'):
            file_path = os.path.join(folderpath, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                process_and_save_json(batch_num, data)
                batch_num+=1
   

# Define the input and output paths
input_path = f"C:\\OneAI\\OneDrive - OneAI\\annotation\\requires_rewrite_only\\annotation_original_batches\\batch_1_src.json"
output_path = 'target.json'


# Process the JSON data and save it
convert_all_json_batches(input_path, output_path)


TypeError: convert_all_json_batches() takes 1 positional argument but 2 were given

In [8]:
import json
import os
"""
DESC: Converted the original format of the batches to the new one that only includes requires rewrite
"""
def process_json(json_data):
    # Read the JSON data from the input file

    # Loop through the data and process as required
    for dialog_key, dialog_data in json_data.items():
        dialog_data['annotator_name'] = None
        annotations = []
        key_to_remove = ['annotator_id', 'number_of_turns']
        for key, value in dialog_data.items():
            # Check if the key is a digit
            if key.isdigit() and key == "1":
                key_to_remove.append(key)
            elif key.isdigit():
                # Add the 'requires_rewrite' key
                annotations.append({'turn_num': int(key),'turn_id': dialog_data['dialog'][int(key)-1]['turn_id'] , 'requires rewrite': None})
                key_to_remove.append(key)
        
        dialog_data['annotations'] = annotations

        for key in key_to_remove:
            del dialog_data[key]

    return json_data


def convert_all_json_batches(folder_path, folder_output_path):
    batch_num = 1
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                json_data = process_json(data)
                output_path = os.path.join(folder_output_path, (f'batch_{batch_num}_src.json')) 
                # Write the modified data to the output file
                with open(output_path, 'w') as file:
                    json.dump(json_data, file, indent=4)
                batch_num+=1
   

# Define the input and output paths
folder_path = f"C:\\OneAI\\OneDrive - OneAI\\annotation\\requires_rewrite_only\\original-batches"
folder_output_path = f"C:\\OneAI\\OneDrive - OneAI\\annotation\\requires_rewrite_only\\requires-rewrite-empty-batches"



# Process the JSON data and save it
convert_all_json_batches(folder_path, folder_output_path)


In [2]:
import json
import os
"""
DESC: Converted the original format of the batches to the new one that only includes requires rewrite
BUT MAKE IT SHORTENED VERSION

"""
def process_json(json_data, dialogs):
    # Read the JSON data from the input file
    new_dialog = {}
    # Loop through the data and process as required
    counter = 0
    for dialog_key, dialog_data in json_data.items():
        
        if counter >= dialogs:
            break
        else:
            counter += 1
            
        dialog_data['annotator_name'] = None
        annotations = []
        key_to_remove = ['annotator_id', 'number_of_turns']
        for key, value in dialog_data.items():
            # Check if the key is a digit
            if key.isdigit() and key == "1":
                key_to_remove.append(key)
            elif key.isdigit():
                # Add the 'requires_rewrite' key
                annotations.append({'turn_num': int(key),'turn_id': dialog_data['dialog'][int(key)-1]['turn_id'] , 'requires rewrite': None})
                key_to_remove.append(key)
        
        dialog_data['annotations'] = annotations

        for key in key_to_remove:
            del dialog_data[key]
            
        new_dialog[dialog_key] = dialog_data

    return new_dialog


def convert_all_json_batches(folder_path, folder_output_path, dialog):
    batch_num = 1
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                json_data = process_json(data, dialog)
                output_path = os.path.join(folder_output_path, (f'batch_{batch_num}_src.json')) 
                # Write the modified data to the output file
                with open(output_path, 'w') as file:
                    json.dump(json_data, file, indent=4)
                batch_num+=1
   

# Define the input and output paths
folder_path = f"C:\\OneAI\\OneDrive - OneAI\\annotation\\requires_rewrite_only\\original-batches"
folder_output_path = f"C:\\OneAI\\OneDrive - OneAI\\annotation\\requires_rewrite_only\\requires-rewrite-empty-batches-shortened"



# Process the JSON data and save it
convert_all_json_batches(folder_path, folder_output_path, dialog=2)
